# Ceneo Scraper
**Struktura opinii w serwisie ceneo.pl:**
|Składowa|Selektor|Zmienna|
|--------|--------|--------|
|*identyfikator opinii*|["data-entry-id"]|`opinion_id`|
|*autora*|span.user-post__author-name|`author`|
|*rekomendację*|span.user-post__author-recomendation > em|`recommend`|
|*liczbę gwiazdek*|span.user-post__score-count|`stars`|
|*treść opinii*|div.user-post__text|`content`|
|*listę wad*|div.review-feature__title review-feature__title--negatives ~ div.review-feature__item|`cons`|
|*listę zalet*|div.review-feature__title review-feature__title--positives ~ div.review-feature__item|`pros`|
|*data wystawienia opinii*|span.user-post__published > time:nth-child(1)["datetime"]|`opinion_date`|
|*data zakupu produktu*|span.user-post__published > time:nth-child(2)["datetime"]|`purchase_date`|
|*ile osób uznało opinię za przydatną*|button.vote-yes["data-total-vote"]|`upvote`|
|*ile osób uznało opinię za nieprzydatną*|button.vote-no["data-total-vote"]|`downvote`|

### 1. Import bibliotek

In [27]:
import requests
from bs4 import BeautifulSoup

### 2. Przypisanie adresu url do zmiennej

In [28]:
product_id = "151210226"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

### 3. Wysłanie do serwera ceneo.pl żądania dostępu do strony z opiniami o produkcie

In [ ]:
response = requests.get(url)
response.status_code

### 4. Etap IV
- Przekształcenie kodu HTML w postaci tekstowej do obiektu reprezentującego strukturę DOM
- Wydobycie ze struktury DOM znaczników odpowiadających opiniom  konsumentów

In [30]:
content = response.text
page = BeautifulSoup(content, "html.parser")
opinion = page.select_one("div.js_product-review")

### 6. Wydobycie z struktury znacznika odpowiadającego pojedynczej opinii jej składowych

In [33]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            return ancestor.select_one(selector)[attribute].strip()
        return ancestor.select_one(selector).text.strip()
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

In [34]:
single_opinion = {
    'opinion_id': extract(opinion, None, "data-entry-id"),
    'author': extract(opinion, "span.user-post__author-name"),
    'recommend': extract(opinion, "span.user-post__author-recomendation > em"),
    'stars': extract(opinion, "span.user-post__score-count"),
    'content': extract(opinion, "div.user-post__text"),
    'cons': extract(opinion, "div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    'pros': extract(opinion, "div.review-feature__title--positives ~ div.review-feature__item", None, True),
    'opinion_date': extract(opinion, "span.user-post__published > time:nth-child(1)", "datetime"),
    'purchase_date': extract(opinion, "span.user-post__published > time:nth-child(2)", "datetime"),
    'upvote': extract(opinion, "button.vote-yes", "data-total-vote"),
    'downvote': extract(opinion, "button.vote-no", "data-total-vote")
}